In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
-import json

with open('math_train.json', 'r', encoding='utf-8') as file:
    train_data = json.load(file)

In [ ]:
import pandas as pd
import json

with open('math_train.json', 'r') as file:
    train_data = json.load(file)

with open('math_test.json', 'r') as file:
    test_data = json.load(file)
train_df = pd.DataFrame(train_data['data'])
test_df = pd.DataFrame(test_data['data'])


In [ ]:
 train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1200 non-null   object
 1   question     1200 non-null   object
 2   choices      1200 non-null   object
 3   explanation  536 non-null    object
 4   answer       1200 non-null   object
dtypes: object(5)
memory usage: 47.0+ KB


In [ ]:
subset_columns = ['question', 'choices', 'explanation', 'answer']
subset_df = train_df[subset_columns]

X = subset_df[['question', 'choices', 'explanation']]
y = subset_df['answer']


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   question     1200 non-null   object
 1   choices      1200 non-null   object
 2   explanation  536 non-null    object
dtypes: object(3)
memory usage: 28.2+ KB


In [ ]:
y.head()

0    D. (80,000 x 6) : 100
1                  C. 8,24
2                D. 1,15m2
3                C. 45 cm2
4               A. 162 cm2
Name: answer, dtype: object

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)

    strategy = tf.distribute.experimental.TPUStrategy
except ValueError:
    strategy = tf.distribute.get_strategy()
    print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
    tpu = None
    gpus = tf.config.experimental.list_logical_devices("GPU")

if tpu:
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu,)
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
    print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy()
    print('Running on single GPU ', gpus[0].name)
else:
    strategy = tf.distribute.get_strategy()
    print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Running on single GPU  /device:GPU:0
Number of accelerators:  1


In [ ]:

!pip3 install transformers

In [ ]:
from transformers import TFAutoModel, AutoTokenizer

In [ ]:
MAX_LEN = 200
model_name = 'vinai/phobert-base'

# Tokenizing
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = [
    tokenizer(
        str(row['question']),
        max_length=MAX_LEN,
        padding='max_length',
        truncation=True,
        return_tensors='np'
    )['input_ids'].reshape(MAX_LEN)
    for index, row in X.iterrows()
]


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.asarray(inputs),
                                                    y,
                                                    random_state=1905,
                                                    test_size=0.2,
                                                    shuffle=True)

In [ ]:
from tensorflow.keras import Input, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Bidirectional, LSTM, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [ ]:
print(y.shape)

(1200,)


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)


In [ ]:
mapping = dict(zip(y_test_encoded, y))

In [ ]:

with strategy.scope():
    encoder = TFAutoModel.from_pretrained(model_name)

    input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32)

    embedding = encoder(input_ids)[1]

    x = Dense(128, activation = 'relu', kernel_regularizer = regularizers.L2(0.1))(embedding)
    x = Dropout(0.2)(x)

    num_classes = len(np.unique(y_train_encoded))

    x = Dense(num_classes,
          activation='softmax',
          kernel_initializer=tf.initializers.GlorotUniform(seed=1905),
          name='output_layer')(x)

    model = Model(inputs=[input_ids], outputs = x)


tf_model.h5:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 tf_roberta_model (TFRobert  TFBaseModelOutputWithPo   134998272 
 aModel)                     olingAndCrossAttentions             
                             (last_hidden_state=(Non             
                             e, 200, 768),                       
                              pooler_output=(None, 7             
                             68),                                
                              past_key_values=None,              
                             hidden_states=None, att             
                             entions=None, cross_att             
                             entions=None)                       
                                                             

In [ ]:
step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-3, 1e-4, 1e-5])

lr = 1e-2 * schedule(step)
wd = lambda: 1e-5 * schedule(step)

In [ ]:
model.compile(
    optimizer=tfa.optimizers.AdamW(weight_decay=wd,
                                   learning_rate=lr),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

In [ ]:

history = model.fit(X_train, y_train_encoded,
                    epochs = 20,
                    verbose = 1,
                    validation_split = 0.1,
                    batch_size = 256)

In [ ]:
pip install transformers datasets evaluate

In [ ]:


train_data = json.load(open('math_train.json', 'r', encoding='utf-8'))['data']
test_data = json.load(open('math_test.json', 'r', encoding='utf-8'))['data']

df_train = pd.DataFrame(train_data)
df_test = pd.DataFrame(test_data)

all_labels = set(df_train['answer'].unique())
unseen_labels = set(df_test['choices'].explode().unique()) - all_labels

if unseen_labels:
    print(f"Unseen labels in test data: {unseen_labels}")
df_test['choices'] = df_test['choices'].apply(lambda x: list(set(x) & all_labels))
df_test = df_test[df_test['choices'].apply(lambda x: bool(x))]

all_labels = set(df_train['answer'].unique())

train, val = train_test_split(df_train, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(train['question'])
X_val = vectorizer.transform(val['question'])

le = preprocessing.LabelEncoder()
y_train = le.fit_transform(train['answer'])
y_val = le.transform(val['answer'])

model = MultinomialNB()
model.fit(X_train, y_train)

predictions = model.predict(X_val)

accuracy = (predictions == y_val).mean()
print(f'Accuracy on validation set: {accuracy}')

X_test = vectorizer.transform(df_test['question'].explode())
test_predictions = model.predict(X_test)

df_test['answer'] = le.inverse_transform(test_predictions)

submission_df = pd.DataFrame({'id': df_test['id'], 'answer': df_test['answer']})
submission_df.to_csv('submission.csv', index=False)


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
ending_names = ["ending0", "ending1", "ending2", "ending3"]
def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples["sent1"]]
    question_headers = examples["sent2"]
    second_sentences = [
        [f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [ ]:
tokenized_swag = swag.map(preprocess_function, batched=True)

Map:   0%|          | 0/73546 [00:00<?, ? examples/s]

Map:   0%|          | 0/20006 [00:00<?, ? examples/s]

Map:   0%|          | 0/20005 [00:00<?, ? examples/s]

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import tensorflow as tf


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="tf",
        )

        batch = {k: tf.reshape(v, (batch_size, num_choices, -1)) for k, v in batch.items()}
        batch["labels"] = tf.convert_to_tensor(labels, dtype=tf.int64)
        return batch

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
model = AutoModelForMultipleChoice.from_pretrained("bert-base-uncased")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_swag_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_swag["train"],
    eval_dataset=tokenized_swag["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

swag = load_dataset("swag", "regular")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/73546 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20006 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20005 [00:00<?, ? examples/s]

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model="EleutherAI/llemma_7b")

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/llemma_7b")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/llemma_7b")


tokenizer_config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

In [ ]:
pip install transformers torch pandas


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch
import pandas as pd
import json


In [ ]:

prediction_indices = torch.argmax(logits, dim=1).tolist()


probs = torch.nn.functional.softmax(logits, dim=1)


predicted_class = torch.argmax(probs, dim=1).tolist()


predicted_choices = [choices[idx] for idx in predicted_class]
bert_predictions.append(predicted_choices)

print(f"Predicted class for example {example['id']}: {predicted_class}")
print(f"Probabilities for example {example['id']}: {probs}")




Predicted class for example 01-0717: [1, 1, 1, 1]
Probabilities for example 01-0717: tensor([[0.4711, 0.5289],
        [0.4767, 0.5233],
        [0.4803, 0.5197],
        [0.4838, 0.5162]], grad_fn=<SoftmaxBackward0>)


In [ ]:

predicted_class = torch.argmax(logits, dim=1).tolist()


In [ ]:
pip install sentencepiece

In [ ]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer
import torch
import pandas as pd
import json

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base')


with open('math_test.json', 'r', encoding='utf-8') as file:
    test_data = json.load(file)['data']

xlm_predictions = []

for example in test_data:
    choices = example['choices'][0] if isinstance(example.get('choices', [''])[0], list) else example.get('choices', [''])

    choices = [str(choice) for choice in choices]
    sequences = [f"{example['question']} {choice}" for choice in choices]
    inputs = tokenizer(sequences, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs) bnm
    logits = outputs.logits
    prediction_indices = torch.argmax(logits, dim=1).tolist()
    predicted_choice = choices[prediction_indices[0]]
    xlm_predictions.append(predicted_choice)

df_results_xlm = pd.DataFrame({'id': [item['id'] for item in test_data], 'answer': xlm_predictions})

df_results_xlm.to_csv('test_predictions_xlm.csv', index=False)



Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:


tokenizer = XLMRobertaTokenizer.from_pretrained('llemma-7b')
model = XLMRobertaForSequenceClassification.from_pretrained('llemma-7b', num_labels=4)

with open('math_train.json', 'r', encoding='utf-8') as file:
    train_data = json.load(file)['data']

train_sequences = [f"{item['question']} {' '.join(item['choices'][0])}" for item in train_data]
train_labels = [ord(item['answer'][0]) - ord('A') for item in train_data]

train_inputs = tokenizer(train_sequences, return_tensors='pt', padding=True, truncation=True)
train_labels = torch.tensor(train_labels)

train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

model.save_pretrained('fine_tuned_llemma-7b')

model = XLMRobertaForSequenceClassification.from_pretrained('fine_tuned_llemma-7b')

with open('math_test.json', 'r', encoding='utf-8') as file:
    test_data = json.load(file)['data']

test_sequences = [f"{item['question']} {' '.join(item['choices'][0])}" for item in test_data]

test_inputs = tokenizer(test_sequences, return_tensors='pt', padding=True, truncation=True)

model.eval()
with torch.no_grad():
    logits = model(**test_inputs).logits

predicted_labels = torch.argmax(logits, dim=1).tolist()

predicted_choices = [chr(label + ord('A')) for label in predicted_labels]

df_results = pd.DataFrame({'id': [item['id'] for item in test_data], 'answer': predicted_choices})
df_results.to_csv('submission.csv', index=False)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
